In [1]:
import pandas as pd
import numpy as np
import requests
import bs4 as bs
import urllib.request

# 2020

In [2]:
link = "https://en.wikipedia.org/wiki/List_of_American_films_of_2020"

In [3]:
source = urllib.request.urlopen(link).read()
soup = bs.BeautifulSoup(source,'lxml')

In [4]:
tables = soup.find_all('table',class_='wikitable sortable')

In [5]:
len(tables)

4

In [6]:
type(tables[0])

bs4.element.Tag

In [7]:
df1 = pd.read_html(str(tables[0]))[0]
df2 = pd.read_html(str(tables[1]))[0]
df3 = pd.read_html(str(tables[2]))[0]
df4 = pd.read_html(str(tables[3]).replace("'1\"\'",'"1"'))[0]

In [10]:
df1["Ref."]=df1[".mw-parser-output .tooltip-dotted{border-bottom:1px dotted;cursor:help}Ref."]
df1=df1.drop([".mw-parser-output .tooltip-dotted{border-bottom:1px dotted;cursor:help}Ref."],axis=1)
df1

,Opening,Opening.1,Title,Production company,Cast and crew,Ref.
0,JANUARY,3,The Grudge,Screen Gems / Stage 6 Films / Ghost House Pict...,Nicolas Pesce (director/screenplay); Andrea Ri...,[2]
1,JANUARY,10,Underwater,20th Century Fox / TSG Entertainment / Chernin...,"William Eubank (director); Brian Duffield, Ada...",[3]
2,JANUARY,10,Like a Boss,Paramount Pictures,"Miguel Arteta (director); Sam Pitman, Adam Col...",[4]
3,JANUARY,10,Three Christs,IFC Films,Jon Avnet (director/screenplay); Eric Nazarian...,NaN
4,JANUARY,10,Inherit the Viper,Barry Films / Tycor International Film Company,Anthony Jerjen (director); Andrew Crabtree (sc...,[5]
5,JANUARY,10,The Sonata,Screen Media Films,Andrew Desmond (director/screenplay); Arthur M...,[6]
6,JANUARY,10,The Murder of Nicole Brown Simpson,Quiver Distribution,Daniel Farrands (director); Michael Arter (scr...,[7]
7,JANUARY,17,Bad Boys for Life,Columbia Pictures / Don Simpson/Jerry Bruckhei...,Adil El Arbi and Bilall Fallah (directors); Ch...,[8]
8,JANUARY,17,Dolittle,Universal Pictures / Perfect World Pictures,Stephen Gaghan (director/screenplay); Dan Greg...,[9]
9,JANUARY,17,A Fall from Grace,Netflix,Tyler Perry (director/screenplay); Crystal R. ...,[10]


In [14]:
df = df1.append(df2.append(df3.append(df4,ignore_index=True),ignore_index=True),ignore_index=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 273 entries, 0 to 272
Data columns (total 6 columns):
Opening               273 non-null object
Opening.1             273 non-null int64
Title                 273 non-null object
Production company    273 non-null object
Cast and crew         273 non-null object
Ref.                  246 non-null object
dtypes: int64(1), object(5)
memory usage: 12.9+ KB


In [12]:
df_2020 = df[['Title','Cast and crew']]

In [13]:
df_2020

,Title,Cast and crew
0,The Grudge,Nicolas Pesce (director/screenplay); Andrea Ri...
1,Underwater,"William Eubank (director); Brian Duffield, Ada..."
2,Like a Boss,"Miguel Arteta (director); Sam Pitman, Adam Col..."
3,Three Christs,Jon Avnet (director/screenplay); Eric Nazarian...
4,Inherit the Viper,Anthony Jerjen (director); Andrew Crabtree (sc...
...,...,...
268,We Can Be Heroes,Robert Rodriguez (director/screenplay); Priyan...
269,News of the World,Paul Greengrass (director/screenplay); Luke Da...
270,One Night in Miami...,Regina King (director); Kemp Powers (screenpla...
271,Promising Young Woman,Emerald Fennell (director/screenplay); Carey M...


In [15]:
from tmdbv3api import TMDb
import json
import requests
tmdb = TMDb()
tmdb.api_key = '70be8e227a430b42ace19b692626b9ce'

In [16]:
from tmdbv3api import Movie
tmdb_movie = Movie() 
def get_genre(x):
    genres = []
    result = tmdb_movie.search(x)
    if not result:
        return np.NaN
    else:
        movie_id = result[0].id
        response = requests.get('https://api.themoviedb.org/3/movie/{}?api_key={}'.format(movie_id,tmdb.api_key))
        data_json = response.json()
        if data_json['genres']:
            genre_str = " " 
            for i in range(0,len(data_json['genres'])):
                genres.append(data_json['genres'][i]['name'])
            return genre_str.join(genres)
        else:
            return np.NaN

In [17]:
df_2020['genres'] = df_2020['Title'].map(lambda x: get_genre(str(x)))

<ipython-input-17-fdc91464740e>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2020['genres'] = df_2020['Title'].map(lambda x: get_genre(str(x)))


In [18]:
df_2020

,Title,Cast and crew,genres
0,The Grudge,Nicolas Pesce (director/screenplay); Andrea Ri...,Horror Mystery Thriller
1,Underwater,"William Eubank (director); Brian Duffield, Ada...",Action Horror Science Fiction Thriller
2,Like a Boss,"Miguel Arteta (director); Sam Pitman, Adam Col...",Comedy
3,Three Christs,Jon Avnet (director/screenplay); Eric Nazarian...,Drama
4,Inherit the Viper,Anthony Jerjen (director); Andrew Crabtree (sc...,Crime Thriller Drama
...,...,...,...
268,We Can Be Heroes,Robert Rodriguez (director/screenplay); Priyan...,Action Fantasy Family Comedy
269,News of the World,Paul Greengrass (director/screenplay); Luke Da...,Adventure Western
270,One Night in Miami...,Regina King (director); Kemp Powers (screenpla...,Drama
271,Promising Young Woman,Emerald Fennell (director/screenplay); Carey M...,Thriller Crime Drama


# 2021

In [20]:
link = "https://en.wikipedia.org/wiki/List_of_American_films_of_2021"

In [21]:
source = urllib.request.urlopen(link).read()
soup = bs.BeautifulSoup(source,'lxml')

In [22]:
tables = soup.find_all('table',class_='wikitable sortable')

In [23]:
len(tables)

4

In [24]:
type(tables[0])

bs4.element.Tag

In [25]:
df1 = pd.read_html(str(tables[0]))[0]
df2 = pd.read_html(str(tables[1]))[0]
df3 = pd.read_html(str(tables[2]))[0]
df4 = pd.read_html(str(tables[3]).replace("'1\"\'",'"1"'))[0]

In [27]:
df1["Ref."]=df1[".mw-parser-output .tooltip-dotted{border-bottom:1px dotted;cursor:help}Ref."]
df1=df1.drop([".mw-parser-output .tooltip-dotted{border-bottom:1px dotted;cursor:help}Ref."],axis=1)
df1

,Opening,Opening.1,Title,Production company,Cast and crew,Ref.
0,JANUARY,1,Shadow in the Cloud,Vertical Entertainment,Roseanne Liang (director/screenplay); Max Land...,[2]
1,JANUARY,13,The White Tiger,Netflix,Ramin Bahrani (director/screenplay); Adarsh Go...,NaN
2,JANUARY,14,Locked Down,HBO Max / Warner Bros. Pictures,Doug Liman (director); Steven Knight (screenpl...,[3]
3,JANUARY,15,The Dig,Netflix / Clerkenwell Films,Simon Stone (director); Moira Buffini (screenp...,[4]
4,JANUARY,15,Outside the Wire,Netflix,"Mikael Håfström (director); Rob Yescombe, Rowa...",[5]
...,...,...,...,...,...,...
59,MARCH,26,Nobody,Universal Pictures / Perfect World Pictures / ...,Ilya Naishuller (director); Derek Kolstad (scr...,[49]
60,MARCH,26,Bad Trip,Netflix / Orion Pictures,Kitao Sakurai (director/screenplay); Dan Curry...,NaN
61,MARCH,26,The Seventh Day,Vertical Entertainment / Voltage Pictures,Justin P. Lange (director/screenplay); Guy Pea...,[50]
62,MARCH,26,Senior Moment,Screen Media Films,"Giorgio Serafini (director); Kurt Brungardt, C...",[51]


In [28]:
df = df1.append(df2.append(df3.append(df4,ignore_index=True),ignore_index=True),ignore_index=True)

In [30]:
df_2021 = df[['Title','Cast and crew']]
df_2021

,Title,Cast and crew
0,Shadow in the Cloud,Roseanne Liang (director/screenplay); Max Land...
1,The White Tiger,Ramin Bahrani (director/screenplay); Adarsh Go...
2,Locked Down,Doug Liman (director); Steven Knight (screenpl...
3,The Dig,Simon Stone (director); Moira Buffini (screenp...
4,Outside the Wire,"Mikael Håfström (director); Rob Yescombe, Rowa..."
...,...,...
343,Sing 2,Garth Jennings (director/screenplay); Matthew ...
344,The King's Man,Matthew Vaughn (director/screenplay); Karl Gaj...
345,The Tragedy of Macbeth,Joel Coen (director/screenplay); Denzel Washin...
346,A Journal for Jordan,Denzel Washington (director); Virgil Williams ...


In [31]:
df_2021['genres'] = df_2021['Title'].map(lambda x: get_genre(str(x)))

<ipython-input-31-252c1cda1e49>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2021['genres'] = df_2021['Title'].map(lambda x: get_genre(str(x)))


In [36]:
df_2020

,Title,Cast and crew,genres
0,The Grudge,Nicolas Pesce (director/screenplay); Andrea Ri...,Horror Mystery Thriller
1,Underwater,"William Eubank (director); Brian Duffield, Ada...",Action Horror Science Fiction Thriller
2,Like a Boss,"Miguel Arteta (director); Sam Pitman, Adam Col...",Comedy
3,Three Christs,Jon Avnet (director/screenplay); Eric Nazarian...,Drama
4,Inherit the Viper,Anthony Jerjen (director); Andrew Crabtree (sc...,Crime Thriller Drama
...,...,...,...
268,We Can Be Heroes,Robert Rodriguez (director/screenplay); Priyan...,Action Fantasy Family Comedy
269,News of the World,Paul Greengrass (director/screenplay); Luke Da...,Adventure Western
270,One Night in Miami...,Regina King (director); Kemp Powers (screenpla...,Drama
271,Promising Young Woman,Emerald Fennell (director/screenplay); Carey M...,Thriller Crime Drama


In [34]:
df = df_2020.append(df_2021,ignore_index=True)

In [35]:
df

,Title,Cast and crew,genres
0,The Grudge,Nicolas Pesce (director/screenplay); Andrea Ri...,Horror Mystery Thriller
1,Underwater,"William Eubank (director); Brian Duffield, Ada...",Action Horror Science Fiction Thriller
2,Like a Boss,"Miguel Arteta (director); Sam Pitman, Adam Col...",Comedy
3,Three Christs,Jon Avnet (director/screenplay); Eric Nazarian...,Drama
4,Inherit the Viper,Anthony Jerjen (director); Andrew Crabtree (sc...,Crime Thriller Drama
...,...,...,...
616,Sing 2,Garth Jennings (director/screenplay); Matthew ...,Animation Comedy Family Music
617,The King's Man,Matthew Vaughn (director/screenplay); Karl Gaj...,War Action Thriller Adventure
618,The Tragedy of Macbeth,Joel Coen (director/screenplay); Denzel Washin...,Drama War
619,A Journal for Jordan,Denzel Washington (director); Virgil Williams ...,Drama Romance


In [37]:
def get_director(x):
    if " (director)" in x:
        return x.split(" (director)")[0]
    elif " (directors)" in x:
        return x.split(" (directors)")[0]
    else:
        return x.split(" (director/screenplay)")[0]

In [38]:
df['director_name'] = df['Cast and crew'].map(lambda x: get_director(str(x)))

In [39]:
def get_actor(x,n):
    if len((x.split("screenplay); ")[-1]).split(", ")) < n:
        return np.NaN
    else:
        return ((x.split("screenplay); ")[-1]).split(", ")[n-1])

In [40]:
df['actor_1_name'] = df['Cast and crew'].map(lambda x: get_actor(x,1))

In [41]:
df['actor_2_name'] = df['Cast and crew'].map(lambda x: get_actor(x,2))

In [42]:
df['actor_3_name'] = df['Cast and crew'].map(lambda x: get_actor(x,3))

In [44]:
df = df.rename(columns={'Title':'movie_title'})

In [45]:
new_df = df.loc[:,['director_name','actor_1_name','actor_2_name','actor_3_name','genres','movie_title']]
new_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 621 entries, 0 to 620
Data columns (total 6 columns):
director_name    621 non-null object
actor_1_name     621 non-null object
actor_2_name     609 non-null object
actor_3_name     569 non-null object
genres           619 non-null object
movie_title      621 non-null object
dtypes: object(6)
memory usage: 29.2+ KB


In [46]:
new_df["actor_2_name"] = new_df["actor_2_name"].replace(np.nan, 'unknown')
new_df["actor_3_name"] = new_df["actor_3_name"].replace(np.nan, 'unknown')

In [47]:
new_df['movie_title'] = new_df['movie_title'].str.lower()

In [48]:
new_df['comb'] = new_df['actor_1_name'] + ' ' + new_df['actor_2_name'] + ' '+ new_df['actor_3_name'] + ' '+ new_df['director_name'] +' ' + new_df['genres']

In [49]:
new_df

,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title,comb
0,Nicolas Pesce,Andrea Riseborough,Demián Bichir,John Cho,Horror Mystery Thriller,the grudge,Andrea Riseborough Demián Bichir John Cho Nico...
1,William Eubank,Kristen Stewart,Vincent Cassel,Jessica Henwick,Action Horror Science Fiction Thriller,underwater,Kristen Stewart Vincent Cassel Jessica Henwick...
2,Miguel Arteta,Tiffany Haddish,Rose Byrne,Salma Hayek,Comedy,like a boss,Tiffany Haddish Rose Byrne Salma Hayek Miguel ...
3,Jon Avnet,Richard Gere,Peter Dinklage,Walton Goggins,Drama,three christs,Richard Gere Peter Dinklage Walton Goggins Jon...
4,Anthony Jerjen,Josh Hartnett,Margarita Levieva,Chandler Riggs,Crime Thriller Drama,inherit the viper,Josh Hartnett Margarita Levieva Chandler Riggs...
...,...,...,...,...,...,...,...
616,Garth Jennings,Matthew McConaughey,Reese Witherspoon,Scarlett Johansson,Animation Comedy Family Music,sing 2,Matthew McConaughey Reese Witherspoon Scarlett...
617,Matthew Vaughn,Ralph Fiennes,Gemma Arterton,Rhys Ifans,War Action Thriller Adventure,the king's man,Ralph Fiennes Gemma Arterton Rhys Ifans Matthe...
618,Joel Coen,Denzel Washington,Frances McDormand,Bertie Carvel,Drama War,the tragedy of macbeth,Denzel Washington Frances McDormand Bertie Car...
619,Denzel Washington,Michael B. Jordan,Chanté Adams,unknown,Drama Romance,a journal for jordan,Michael B. Jordan Chanté Adams unknown Denzel ...


In [50]:
old_df = pd.read_csv('final_data.csv')

In [51]:
old_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5874 entries, 0 to 5873
Data columns (total 7 columns):
director_name    5874 non-null object
actor_1_name     5874 non-null object
actor_2_name     5874 non-null object
actor_3_name     5874 non-null object
genres           5874 non-null object
movie_title      5874 non-null object
comb             5874 non-null object
dtypes: object(7)
memory usage: 321.4+ KB


In [52]:
final_df = old_df.append(new_df,ignore_index=True)

In [53]:
final_df

,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title,comb
0,James Cameron,CCH Pounder,Joel David Moore,Wes Studi,Action Adventure Fantasy Sci-Fi,avatar,CCH Pounder Joel David Moore Wes Studi James C...
1,Gore Verbinski,Johnny Depp,Orlando Bloom,Jack Davenport,Action Adventure Fantasy,pirates of the caribbean: at world's end,Johnny Depp Orlando Bloom Jack Davenport Gore ...
2,Sam Mendes,Christoph Waltz,Rory Kinnear,Stephanie Sigman,Action Adventure Thriller,spectre,Christoph Waltz Rory Kinnear Stephanie Sigman ...
3,Christopher Nolan,Tom Hardy,Christian Bale,Joseph Gordon-Levitt,Action Thriller,the dark knight rises,Tom Hardy Christian Bale Joseph Gordon-Levitt ...
4,Doug Walker,Doug Walker,Rob Walker,unknown,Documentary,star wars: episode vii - the force awakens ...,Doug Walker Rob Walker unknown Doug Walker Doc...
...,...,...,...,...,...,...,...
6490,Garth Jennings,Matthew McConaughey,Reese Witherspoon,Scarlett Johansson,Animation Comedy Family Music,sing 2,Matthew McConaughey Reese Witherspoon Scarlett...
6491,Matthew Vaughn,Ralph Fiennes,Gemma Arterton,Rhys Ifans,War Action Thriller Adventure,the king's man,Ralph Fiennes Gemma Arterton Rhys Ifans Matthe...
6492,Joel Coen,Denzel Washington,Frances McDormand,Bertie Carvel,Drama War,the tragedy of macbeth,Denzel Washington Frances McDormand Bertie Car...
6493,Denzel Washington,Michael B. Jordan,Chanté Adams,unknown,Drama Romance,a journal for jordan,Michael B. Jordan Chanté Adams unknown Denzel ...


In [54]:
final_df.isnull().sum()

director_name    0
actor_1_name     0
actor_2_name     0
actor_3_name     0
genres           2
movie_title      0
comb             2
dtype: int64

In [55]:
final_df = final_df.dropna(how='any')

In [58]:
final_df.to_csv('main_data.csv',index=False)

In [59]:
final_df

,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title,comb
0,James Cameron,CCH Pounder,Joel David Moore,Wes Studi,Action Adventure Fantasy Sci-Fi,avatar,CCH Pounder Joel David Moore Wes Studi James C...
1,Gore Verbinski,Johnny Depp,Orlando Bloom,Jack Davenport,Action Adventure Fantasy,pirates of the caribbean: at world's end,Johnny Depp Orlando Bloom Jack Davenport Gore ...
2,Sam Mendes,Christoph Waltz,Rory Kinnear,Stephanie Sigman,Action Adventure Thriller,spectre,Christoph Waltz Rory Kinnear Stephanie Sigman ...
3,Christopher Nolan,Tom Hardy,Christian Bale,Joseph Gordon-Levitt,Action Thriller,the dark knight rises,Tom Hardy Christian Bale Joseph Gordon-Levitt ...
4,Doug Walker,Doug Walker,Rob Walker,unknown,Documentary,star wars: episode vii - the force awakens ...,Doug Walker Rob Walker unknown Doug Walker Doc...
...,...,...,...,...,...,...,...
6490,Garth Jennings,Matthew McConaughey,Reese Witherspoon,Scarlett Johansson,Animation Comedy Family Music,sing 2,Matthew McConaughey Reese Witherspoon Scarlett...
6491,Matthew Vaughn,Ralph Fiennes,Gemma Arterton,Rhys Ifans,War Action Thriller Adventure,the king's man,Ralph Fiennes Gemma Arterton Rhys Ifans Matthe...
6492,Joel Coen,Denzel Washington,Frances McDormand,Bertie Carvel,Drama War,the tragedy of macbeth,Denzel Washington Frances McDormand Bertie Car...
6493,Denzel Washington,Michael B. Jordan,Chanté Adams,unknown,Drama Romance,a journal for jordan,Michael B. Jordan Chanté Adams unknown Denzel ...
